In [1]:
import pandas as pd

import pyarrow.parquet as pq
from pyarrow.parquet import ParquetFile
import pyarrow as pa 
from sklearn.utils import resample
from imblearn.under_sampling import RandomUnderSampler
from imblearn.under_sampling import TomekLinks
import tqdm
from imblearn.under_sampling import CondensedNearestNeighbour
import datetime

In [2]:
parquet_file = pq.ParquetFile('../../input_data/train.parquet')

batch_size = 115_000_000
rows = next(parquet_file.iter_batches(batch_size=batch_size)) 
df = rows.to_pandas()  
rows=0
df

,user_id,adv_campaign_id,platform_id,adv_creative_id,event_date,banner_code,is_main,target
0,2853707,3352,3,3075,2024-09-17,6,True,0
1,2537244,4029,2,3260,2024-09-17,8,True,0
2,63033,1578,3,1109,2024-09-17,6,True,0
3,164702,3434,1,1079,2024-09-17,7,True,0
4,2802905,2208,3,3576,2024-09-17,6,True,0
...,...,...,...,...,...,...,...,...
114741030,2670849,3904,2,3275,2024-09-08,8,True,0
114741031,3187004,2142,2,238,2024-09-08,8,True,0
114741032,821310,2414,2,2654,2024-09-08,8,True,0
114741033,2282258,3120,2,551,2024-09-08,5,False,1


In [3]:
parquet_file = pq.ParquetFile('../../input_data/test.parquet')

batch_size = 115_000_000
rows = next(parquet_file.iter_batches(batch_size=batch_size)) 
test = rows.to_pandas()  
rows=0
test

,user_id,adv_campaign_id,platform_id,adv_creative_id,event_date,banner_code,is_main
0,2714742,3026,2,4056,2024-09-23,8,True
1,2714742,2994,2,3954,2024-09-23,8,True
2,2714742,97,2,1752,2024-09-23,8,True
3,2714742,3539,2,1244,2024-09-23,8,True
4,2714742,2756,2,2003,2024-09-23,8,True
...,...,...,...,...,...,...,...
1983282,2398626,1099,2,3030,2024-09-23,8,True
1983283,2398626,488,2,1324,2024-09-23,8,True
1983284,2398626,49,2,1514,2024-09-23,8,True
1983285,2398626,1053,2,4211,2024-09-23,5,False


# формируем выборку только по creative_id, встречающимся в тесте, для них берем все лайки и рандомно отбираем часть игноров

In [4]:
idx = test.adv_creative_id.unique()
len(test.adv_creative_id.unique()), len(df.adv_creative_id.unique())

(2589, 4291)

In [5]:
df0 = df[df.adv_creative_id.isin(idx)]

In [6]:
df0.target.value_counts()

target
0    81456972
1      442552
Name: count, dtype: int64

In [7]:
df0_1 = df0[df0.target==1]
df0_0 = df0[df0.target==0]

df0_0 = df0_0.sample(frac=0.10, replace=False, random_state=1)

In [8]:
data = pd.concat([df0_1, df0_0], axis=0)
tl = TomekLinks(sampling_strategy='majority')
X, y = tl.fit_resample(data.drop(columns=['target', 'event_date']), data.target)

In [9]:
X

,user_id,adv_campaign_id,platform_id,adv_creative_id,banner_code,is_main
0,745232,3063,2,330,8,True
1,726464,1494,2,719,8,True
2,820269,2606,2,137,5,False
3,2445074,1857,3,2202,6,True
4,1425141,3820,3,4064,6,True
...,...,...,...,...,...,...
8384232,2882454,570,2,1588,8,True
8384233,2496359,1303,2,1960,8,True
8384234,1693916,1663,2,2282,8,True
8384235,1228439,3890,3,2244,6,True


In [10]:
date = data.event_date.iloc[tl.sample_indices_].reset_index().drop(columns='index')
date

,event_date
0,2024-09-17
1,2024-09-17
2,2024-09-17
3,2024-09-17
4,2024-09-17
...,...
8384232,2024-09-19
8384233,2024-09-15
8384234,2024-09-17
8384235,2024-09-22


In [11]:
X2 = pd.concat([X, date], axis=1) 
X2 = pd.concat([X2, y], axis=1)        
X2

,user_id,adv_campaign_id,platform_id,adv_creative_id,banner_code,is_main,event_date,target
0,745232,3063,2,330,8,True,2024-09-17,1
1,726464,1494,2,719,8,True,2024-09-17,1
2,820269,2606,2,137,5,False,2024-09-17,1
3,2445074,1857,3,2202,6,True,2024-09-17,1
4,1425141,3820,3,4064,6,True,2024-09-17,1
...,...,...,...,...,...,...,...,...
8384232,2882454,570,2,1588,8,True,2024-09-19,0
8384233,2496359,1303,2,1960,8,True,2024-09-15,0
8384234,1693916,1663,2,2282,8,True,2024-09-17,0
8384235,1228439,3890,3,2244,6,True,2024-09-22,0


In [12]:
X2.to_csv('creative_resample_f.csv', sep=',', index=False)